In [1]:
import json
import numpy as np
import networkx as nx
import tqdm
from sys import argv
import os
import yaml
import pandas as pd
import csv

In [2]:
input_name = "./data/reindexed.json"
output_dir = "./data"
output_name = "./data/partition/graph-for-metis.txt"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"'{output_dir}' created.")
else:
    print(
        f"'{output_dir}' already exists. Will add switch configs there...")


'./data' already exists. Will add switch configs there...


In [3]:
# read topo data
with open(input_name, 'r') as f:
    data = json.load(f)

In [4]:

# get nodes and egdes
nodes, edges = data['nodeList'], data['edgeList']
NUM_NODES, NUM_EDGES = len(nodes), len(edges)

# creating some usefull maps to get information regarding the nodes
new_to_old = {}
for node in nodes:
    new_to_old[node['id']] = node['old_id']

id_to_node = {}
for node in nodes:
    id_to_node[node['id']] = node

port_map = {}
for node in nodes:
    port_map[node['id']] = {}
    for port in node['portList']:
        port_map[node['id']][port['id']] = port

######################################################################################
#                                PARSING
######################################################################################

print('Parsing data and creating graph...')

# the topology adj. matrix
adj = np.zeros((NUM_NODES, NUM_NODES))
# connection details for topology
connection_info = {node['id']: {} for node in nodes}

# for each node find all edges that start from that node and add them to the adj. matrix
for node in nodes:
    for edge in edges:
        if node['id'] == edge['srcNode']:
            src, src_port, dest, dest_port = node['id'], edge['srcPort'], edge['destNode'], edge['destPort']
            if adj[src, dest] == 0:  # NOTE: FOR NOW, IGNORING DUPLICATE EDGES CONNECTIONS
                adj[src, dest] = 1  # adj. matrix
                adj[dest, src] = 1
                # Getting src node port info
                for p in id_to_node[src]['portList']:
                    if p['id'] == src_port:
                        connection_info[src][dest] = p



Parsing data and creating graph...


In [5]:
# remove edges linking to the node itself
for i in range(len(adj)):
    adj[i,i] = 0


In [6]:

######################################################################################
#                                CREATING GRAPHS
######################################################################################

# Create a undirected graph from the dictionary
g = nx.from_numpy_array(adj, create_using=nx.DiGraph)

# get the nodes of the subgraphs in the topology
subgraphs = list(nx.strongly_connected_components(g))

for idx, cc in enumerate(subgraphs):
    print(f'subgraph index: {idx}: nodes in subgraph {len(cc)}')

index = int(input("Choose subgraph: "))

# get the topology of selected subgraph
g = g.subgraph(subgraphs[index])

subgraph index: 0: nodes in subgraph 5237
subgraph index: 1: nodes in subgraph 1703
subgraph index: 2: nodes in subgraph 2039
Choose subgraph: 0


In [ ]:
#####################################################
# PARTITIONING WITH NO WEIGHTS OR SIZES
#####################################################

In [7]:
with open(output_name, 'w') as f:
    f.write(str(len(g.nodes)) + " {}\n".format(int(g.number_of_edges()/2)))
    print(len(g.nodes), int(g.number_of_edges()/2))
    node_list = sorted(g.nodes)
    for node in node_list:
        neighbors = list(g.neighbors(node))
        neighbors = [i+1 for i in neighbors]
        str_neighbors = ' '.join(str(ngh) for ngh in neighbors)
        print(f"Neighbors of Node {node+1}: {str_neighbors}")
        f.write(str_neighbors + "\n")


5237 6067
Neighbors of Node 1: 550 3191
Neighbors of Node 2: 3361 4506
Neighbors of Node 3: 319 4343
Neighbors of Node 4: 1287 2324
Neighbors of Node 5: 3934 5110
Neighbors of Node 6: 3774 4889
Neighbors of Node 7: 1719 2130
Neighbors of Node 8: 1693
Neighbors of Node 9: 218
Neighbors of Node 10: 3398 4417
Neighbors of Node 11: 2952 4773
Neighbors of Node 12: 1386 4295
Neighbors of Node 13: 297 1575
Neighbors of Node 14: 509 1486
Neighbors of Node 15: 365 4180
Neighbors of Node 16: 172 2179
Neighbors of Node 17: 741 822
Neighbors of Node 18: 2024 3386
Neighbors of Node 19: 2028 3513
Neighbors of Node 20: 720 2776
Neighbors of Node 21: 4664
Neighbors of Node 22: 392 1526 1926 2246 2575 3213 3912 4026 4044 5191
Neighbors of Node 23: 1271 4454
Neighbors of Node 24: 265 2994
Neighbors of Node 25: 3004 3298
Neighbors of Node 26: 2610 3127
Neighbors of Node 27: 3593 4287
Neighbors of Node 28: 886 3064
Neighbors of Node 29: 2190 2838 4655 4769
Neighbors of Node 30: 1203 2439
Neighbors of Node

In [50]:
def read_traces(traces_file):
    traces_df = pd.read_csv(traces_file, sep=' ',header=None)
    assert(traces_df[5][0]=="R" and traces_df[7][0]=="S")
    assert(traces_df[0][5237] == "Terminal")
    return traces_df

In [ ]:
#################################
# Partition with Vetex Weights 1. -- number of events
#################################

In [57]:
traces_e_file = "./data/traces-detail-events.txt"
traces_e_df = read_traces(traces_e_file)
traces_e_df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Access,Switch,0,final_dest,0.0,R,2736.0,S,2736.0,D,0.0,|549|2736,2736.0
1,Access,Switch,1,final_dest,0.0,R,1738.0,S,1738.0,D,0.0,|3360|1738,1738.0
2,Access,Switch,2,final_dest,0.0,R,3641.0,S,3641.0,D,0.0,|4342|3641,3641.0
3,Access,Switch,3,final_dest,0.0,R,2708.0,S,2708.0,D,0.0,|1286|2708,2708.0
4,Access,Switch,4,final_dest,0.0,R,871.0,S,871.0,D,0.0,|3933|871,871.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5233,Access,Switch,5233,final_dest,0.0,R,1799.0,S,1799.0,D,0.0,|4862|1799,1799.0
5234,Access,Switch,5234,final_dest,0.0,R,1830.0,S,1830.0,D,0.0,|549|1830,1830.0
5235,Access,Switch,5235,final_dest,0.0,R,865.0,S,865.0,D,0.0,|1581|865,865.0
5236,Access,Switch,5236,final_dest,0.0,R,4390.0,S,4390.0,D,0.0,|1084|4390,4390.0


In [72]:
events_df = traces_e_df[[2,12]][:-1]
events_df.columns = ["id","events"]
events_df = events_df.astype('int')
events_df.sort_values(by="id", inplace=True)
events_df

,id,events
0,0,2736
1,1,1738
2,2,3641
3,3,2708
4,4,871
...,...,...
5232,5232,867
5233,5233,1799
5234,5234,1830
5235,5235,865


In [74]:
events_df.iloc[2]['events']

3641

In [77]:
output_name = "./data/partition/graph-for-metis-w-vertex-weight.txt"
with open(output_name, 'w') as f:
    f.write(str(len(g.nodes)) + " {} 010\n".format(int(g.number_of_edges()/2)))
    print(len(g.nodes), int(g.number_of_edges()/2))
    node_list = sorted(g.nodes)
    i = 0
    for node in node_list:
        n_events = events_df.iloc[i]['events']
        index = events_df.iloc[i]['id']
        assert(node == index)
        neighbors = list(g.neighbors(node))
        neighbors = [i+1 for i in neighbors]
        str_neighbors = str(n_events) + ' ' + ' '.join(str(ngh) for ngh in neighbors)
        print(f"Neighbors of Node {node+1} [{n_events}]: {str_neighbors}")
        f.write(str_neighbors + "\n")
        i = i + 1


5237 6067
Neighbors of Node 1 [2736]: 2736 550 3191
Neighbors of Node 2 [1738]: 1738 3361 4506
Neighbors of Node 3 [3641]: 3641 319 4343
Neighbors of Node 4 [2708]: 2708 1287 2324
Neighbors of Node 5 [871]: 871 3934 5110
Neighbors of Node 6 [1816]: 1816 3774 4889
Neighbors of Node 7 [2685]: 2685 1719 2130
Neighbors of Node 8 [887]: 887 1693
Neighbors of Node 9 [886]: 886 218
Neighbors of Node 10 [3552]: 3552 3398 4417
Neighbors of Node 11 [2716]: 2716 2952 4773
Neighbors of Node 12 [1748]: 1748 1386 4295
Neighbors of Node 13 [5420]: 5420 297 1575
Neighbors of Node 14 [5432]: 5432 509 1486
Neighbors of Node 15 [2683]: 2683 365 4180
Neighbors of Node 16 [1824]: 1824 172 2179
Neighbors of Node 17 [888]: 888 741 822
Neighbors of Node 18 [1789]: 1789 2024 3386
Neighbors of Node 19 [3619]: 3619 2028 3513
Neighbors of Node 20 [1834]: 1834 720 2776
Neighbors of Node 21 [864]: 864 4664
Neighbors of Node 22 [17187]: 17187 392 1526 1926 2246 2575 3213 3912 4026 4044 5191
Neighbors of Node 23 [178

In [ ]:
#################################
# Partition with Vetex Weights 2. -- number of received packets
#################################

In [78]:
traces_p_file = "./data/traces-detail-events.txt"
traces_p_df = read_traces(traces_p_file)
traces_p_df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Access,Switch,0,final_dest,0.0,R,2736.0,S,2736.0,D,0.0,|549|2736,2736.0
1,Access,Switch,1,final_dest,0.0,R,1738.0,S,1738.0,D,0.0,|3360|1738,1738.0
2,Access,Switch,2,final_dest,0.0,R,3641.0,S,3641.0,D,0.0,|4342|3641,3641.0
3,Access,Switch,3,final_dest,0.0,R,2708.0,S,2708.0,D,0.0,|1286|2708,2708.0
4,Access,Switch,4,final_dest,0.0,R,871.0,S,871.0,D,0.0,|3933|871,871.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5233,Access,Switch,5233,final_dest,0.0,R,1799.0,S,1799.0,D,0.0,|4862|1799,1799.0
5234,Access,Switch,5234,final_dest,0.0,R,1830.0,S,1830.0,D,0.0,|549|1830,1830.0
5235,Access,Switch,5235,final_dest,0.0,R,865.0,S,865.0,D,0.0,|1581|865,865.0
5236,Access,Switch,5236,final_dest,0.0,R,4390.0,S,4390.0,D,0.0,|1084|4390,4390.0


In [82]:
packets_df = traces_p_df[[2,4,6]][:-1]
packets_df.columns = ["id","final_hop","recv"]
packets_df = packets_df.astype('int')
packets_df.sort_values(by="id", inplace=True)
packets_df['weight']=packets_df['final_hop'] + packets_df['recv']
packets_df

,id,final_hop,recv,weight
0,0,0,2736,2736
1,1,0,1738,1738
2,2,0,3641,3641
3,3,0,2708,2708
4,4,0,871,871
...,...,...,...,...
5232,5232,0,867,867
5233,5233,0,1799,1799
5234,5234,0,1830,1830
5235,5235,0,865,865


In [83]:
output_name = "./data/partition/graph-for-metis-w-vertex-weight-pkt.txt"
with open(output_name, 'w') as f:
    f.write(str(len(g.nodes)) + " {} 010\n".format(int(g.number_of_edges()/2)))
    print(len(g.nodes), int(g.number_of_edges()/2))
    node_list = sorted(g.nodes)
    i = 0
    for node in node_list:
        weight = packets_df.iloc[i]['weight']
        index = packets_df.iloc[i]['id']
        assert(node == index)
        neighbors = list(g.neighbors(node))
        neighbors = [i+1 for i in neighbors]
        str_neighbors = str(weight) + ' ' + ' '.join(str(ngh) for ngh in neighbors)
        print(f"Neighbors of Node {node+1} [{weight}]: {str_neighbors}")
        f.write(str_neighbors + "\n")
        i = i + 1


5237 6067
Neighbors of Node 1 [2736]: 2736 550 3191
Neighbors of Node 2 [1738]: 1738 3361 4506
Neighbors of Node 3 [3641]: 3641 319 4343
Neighbors of Node 4 [2708]: 2708 1287 2324
Neighbors of Node 5 [871]: 871 3934 5110
Neighbors of Node 6 [1816]: 1816 3774 4889
Neighbors of Node 7 [2685]: 2685 1719 2130
Neighbors of Node 8 [887]: 887 1693
Neighbors of Node 9 [886]: 886 218
Neighbors of Node 10 [3552]: 3552 3398 4417
Neighbors of Node 11 [2716]: 2716 2952 4773
Neighbors of Node 12 [1748]: 1748 1386 4295
Neighbors of Node 13 [5420]: 5420 297 1575
Neighbors of Node 14 [5432]: 5432 509 1486
Neighbors of Node 15 [2683]: 2683 365 4180
Neighbors of Node 16 [1824]: 1824 172 2179
Neighbors of Node 17 [888]: 888 741 822
Neighbors of Node 18 [1789]: 1789 2024 3386
Neighbors of Node 19 [3619]: 3619 2028 3513
Neighbors of Node 20 [1834]: 1834 720 2776
Neighbors of Node 21 [864]: 864 4664
Neighbors of Node 22 [17107]: 17107 392 1526 1926 2246 2575 3213 3912 4026 4044 5191
Neighbors of Node 23 [178

In [ ]:
#################################
# PARTITIONING WITH EDGE WEIGHTS
#################################

In [54]:
traces_file = "./data/traces_detail.txt"
traces_df = read_traces(traces_file)
traces_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,Access,Switch,0,final_dest,0.0,R,2736.0,S,2736.0,D,0.0,|549|2736
1,Access,Switch,1,final_dest,0.0,R,1738.0,S,1738.0,D,0.0,|3360|1738
2,Access,Switch,2,final_dest,0.0,R,3641.0,S,3641.0,D,0.0,|4342|3641
3,Access,Switch,3,final_dest,0.0,R,2708.0,S,2708.0,D,0.0,|1286|2708
4,Access,Switch,4,final_dest,0.0,R,871.0,S,871.0,D,0.0,|3933|871
...,...,...,...,...,...,...,...,...,...,...,...,...
5233,Access,Switch,5233,final_dest,0.0,R,1799.0,S,1799.0,D,0.0,|4862|1799
5234,Access,Switch,5234,final_dest,0.0,R,1830.0,S,1830.0,D,0.0,|549|1830
5235,Access,Switch,5235,final_dest,0.0,R,865.0,S,865.0,D,0.0,|1581|865
5236,Access,Switch,5236,final_dest,0.0,R,4390.0,S,4390.0,D,0.0,|1084|4390


In [47]:
# remove terminal info
send_all_df = traces_df[[2, 8, 6, 11]][:-1]
send_all_df[[2, 8, 6]] = send_all_df[[2, 8, 6]].astype(int)
send_all_df.columns = ["id", "send_count", "recv_count", "send_per_edge"]
send_all_df = send_all_df.sort_values(by='id')

send_all_df.fillna('', inplace=True)
send_all_df

,id,send_count,recv_count,send_per_edge
0,0,2736,2736,|549|2736
1,1,1738,1738,|3360|1738
2,2,3641,3641,|4342|3641
3,3,2708,2708,|1286|2708
4,4,871,871,|3933|871
...,...,...,...,...
5232,5232,867,867,|3044|867
5233,5233,1799,1799,|4862|1799
5234,5234,1830,1830,|549|1830
5235,5235,865,865,|1581|865


In [48]:
send_str = send_all_df['send_per_edge']
indexes = send_all_df['id']
count_per_edge = []
for i in range(len(send_str)): 
    src = indexes[i]
#      print(i, send_str[i])
    tmp_data = np.array(send_str[i].split('|')[1:])
    tmp_data_reshape = np.reshape(tmp_data, (-1, 2))
#     if send_str[i] == '':
#         print(tmp_data_reshape)
    #print(src, tmp_data_reshape)
    for item in tmp_data_reshape:
        count_per_edge.append([i, int(item[0]), int(item[1])])  # [src, dest, send_count]

count_per_edge_df = pd.DataFrame(count_per_edge, columns=['src', 'dest', 'send_count'])
count_per_edge_df

,src,dest,send_count
0,0,549,2736
1,1,3360,1738
2,2,4342,3641
3,3,1286,2708
4,4,3933,871
...,...,...,...
5561,5232,3044,867
5562,5233,4862,1799
5563,5234,549,1830
5564,5235,1581,865


In [49]:
tmp_csv_file = "./data/partition/w-weights.csv"
copy_df = count_per_edge_df.copy()
copy_df.columns=['dest','src','send_count2']
copy_df = copy_df[['src','dest','send_count2']]
count_merged_df = pd.merge(count_per_edge_df, copy_df, on=['src','dest'], how='outer').fillna(0)
count_merged_df['total'] = (count_merged_df['send_count'] + count_merged_df['send_count2']).astype('int')
count_merged_df.sort_values(by=['src','dest'],inplace=True)
count_merged_df.to_csv(tmp_csv_file, index=False)
count_merged_df['total_scaled'] = (count_merged_df['total']/400).astype('int')
count_merged_df

,src,dest,send_count,send_count2,total,total_scaled
0,0,549,2736.0,0.0,2736,6
8709,0,3190,0.0,1795.0,1795,4
1,1,3360,1738.0,0.0,1738,4
10003,1,4505,0.0,892.0,892,2
5879,2,318,0.0,2720.0,2720,6
...,...,...,...,...,...,...
5563,5234,549,1830.0,0.0,1830,4
8293,5234,2768,0.0,913.0,913,2
5564,5235,1581,865.0,0.0,865,2
5565,5236,1084,4390.0,0.0,4390,10


In [43]:
print(count_merged_df['total'].min())
print(count_merged_df['total'].max())
print(count_merged_df['total_scaled'].min())
print(count_merged_df['total_scaled'].max())

807
1678634
2
4196


10728

In [44]:
# Write to file
output_name_w_weight = "./data/partition/graph-for-metis-w-weight.txt"
column_name = 'total'

custom_header = str(len(g.nodes)) + " {} 001\n".format(int(len(count_merged_df)/2))
with open(output_name_w_weight, 'w') as file:
    print(custom_header)
    file.write(custom_header)

    line = ""
    src = count_merged_df['src'][0]
    print(src)
    for index, row in count_merged_df.iterrows():
        if src != row['src']:
            # now we are on the line of a new src ID, then flush previous line to file
            line = line.strip()
            print("[{}]".format(int(src)), line)
            file.write(line + '\n')
            src = row['src']
            line = ""
        line = line + str(int(row['dest'])+1) + " " +  str(int(row[column_name])) + " "
    # need to flush the final line
    print("[{}]".format(int(src)), line)
    file.write(line + '\n')

5237 5364 001

0
[0] 550 2736 3191 1795
[1] 3361 1738 4506 892
[2] 319 2720 4343 3641
[3] 1287 2708 2324 1802
[4] 3934 871
[5] 3774 1816 4889 933
[6] 1719 1777 2130 2685
[7] 1693 887
[8] 218 886
[9] 3398 2658 4417 3552
[10] 2952 1819 4773 2716
[11] 1386 883 4295 1748
[12] 297 5420 1575 4495
[13] 509 4530 1486 5432
[14] 365 1783 4180 2683
[15] 172 1824 2179 906
[16] 822 888
[17] 2024 1789 3386 889
[18] 2028 3619 3513 2733
[19] 720 1834 2776 919
[20] 4664 864
[21] 392 1833 1526 17107 1926 1817 2246 891 2575 3574 3213 1769 3912 925 4026 1808 4044 1799 5191 1814
[22] 1271 1786 4454 887
[23] 265 901 2994 1798
[24] 3298 927
[25] 2610 3583 3127 2667
[26] 3593 896
[27] 3064 890
[28] 2838 1775 4655 7156 4769 4508
[29] 1203 907 2439 1777
[30] 1977 889
[31] 221 831
[32] 2319 2646 3366 1746
[33] 3050 3585 5156 4486
[34] 3300 2659 3669 1774
[35] 1096 1795 1619 2666
[36] 773 4402 2318 5329
[37] 1164 1722 2690 881
[38] 2278 907
[39] 371 2660 4751 1738
[40] 1694 1812 4056 2686
[41] 3213 858
[42] 1419 

In [ ]:
#################################
# PARTITIONING WITH VERTEX SIZE
#################################
# size_of_each_vertex = send_count/edge_count


In [201]:
# remove terminal info
send_df = traces_df[[2, 8, 6]][:-1]
send_df.columns = ["id", "send_count", "recv_count"]
send_df = send_df.astype(int)
send_df = send_df.sort_values(by='id')
send_df

,id,send_count,recv_count
0,0,2736,2736
1,1,1738,1738
2,2,3641,3641
3,3,2708,2708
4,4,871,871
...,...,...,...
5232,5232,867,867
5233,5233,1799,1799
5234,5234,1830,1830
5235,5235,865,865


In [156]:
# Prepare edges data 
node_list = sorted(g.nodes)
# here we need to plus one to align with the numbering in METIS
tmp = [[node, len(list(g.neighbors(node))), ' '.join(str(ngh+1) for ngh in g.neighbors(node))] for node in node_list]
edge_df = pd.DataFrame(tmp, columns=['id', 'num_edges', 'edges+1'])
edge_df
# for node in node_list:
#     neighbors = list(g.neighbors(node))
#     print(len(neighbors), neighbors)

,id,num_edges,edges+1
0,0,2,550 3191
1,1,2,3361 4506
2,2,2,319 4343
3,3,2,1287 2324
4,4,2,3934 5110
...,...,...,...
5232,5232,2,2735 3045
5233,5233,2,3528 4863
5234,5234,2,550 2769
5235,5235,2,1582 3525


In [214]:
edge_df.describe()

,id,num_edges
count,5237.000000,5237.000000
mean,2618.000000,2.316975
std,1511.936011,1.609857
min,0.000000,1.000000
25%,1309.000000,2.000000
50%,2618.000000,2.000000
75%,3927.000000,2.000000
max,5236.000000,25.000000


In [206]:
# Calculate sizes
merged_df = pd.merge(send_df, edge_df, on='id')
scaling_factor = 200
merged_df['vsize']= (merged_df['send_count']+merged_df['recv_count'])/merged_df['num_edges']


print("min(exclude 0):", merged_df['vsize'][merged_df['vsize']!=0].min())
print("max:", merged_df['vsize'].max())
assert(merged_df['vsize'][merged_df['vsize']!=0].min() > scaling_factor)

merged_df['vsize_int'] = merged_df['vsize'].astype('int')
merged_df['vsize_scaled'] = (merged_df['vsize']/scaling_factor).astype('int')
merged_df

min(exclude 0): 594.0
max: 432993.0


,id,send_count,recv_count,num_edges,edges+1,vsize,vsize_int,vsize_scaled
0,0,2736,2736,2,550 3191,2736.0,2736,13
1,1,1738,1738,2,3361 4506,1738.0,1738,8
2,2,3641,3641,2,319 4343,3641.0,3641,18
3,3,2708,2708,2,1287 2324,2708.0,2708,13
4,4,871,871,2,3934 5110,871.0,871,4
...,...,...,...,...,...,...,...,...
5232,5232,867,867,2,2735 3045,867.0,867,4
5233,5233,1799,1799,2,3528 4863,1799.0,1799,8
5234,5234,1830,1830,2,550 2769,1830.0,1830,9
5235,5235,865,865,2,1582 3525,865.0,865,4


In [213]:
merged_df['vsize_scaled'].describe()

count    5237.000000
mean       18.248998
std        66.181241
min         0.000000
25%         8.000000
50%         9.000000
75%        17.000000
max      2164.000000
Name: vsize_scaled, dtype: float64

In [208]:
# Write to file
output_name_w_size = "./data/partition/graph-for-metis-w-size-scaled-sendrecv.txt"
column_name = 'vsize_scaled'

custom_header = str(len(g.nodes)) + " {} 100\n".format(int(g.number_of_edges()/2))
with open(output_name_w_size, 'w') as file:
    print(custom_header)
    file.write(custom_header)

# Append DataFrame to the CSV file
merged_df[[column_name, 'edges+1']].to_csv(output_name_w_size, sep=',', mode='a', index=False, header=False, quoting=csv.QUOTE_NONE)


# Then replace all ',' to ' '
with open(output_name_w_size, 'r') as file:
    content = file.read()
# Replace commas with spaces
modified_content = content.replace(',', ' ')
# Write the modified content back to the file
with open(output_name_w_size, 'w') as file:
    file.write(modified_content)

# merged_df[['vsize_scaled', 'edges+1']]

5237 6067 100



In [108]:


output_name_w_size = "./data/partition/graph-for-metis-size.txt"
with open(output_name_w_size, 'w') as f:
    # write the header
    f.write(str(len(g.nodes)) + " {} 100\n".format(int(g.number_of_edges()/2)))
    print(len(g.nodes), int(g.number_of_edges()/2))
    node_list = sorted(g.nodes)
    for node in node_list:
        neighbors = list(g.neighbors(node))
        neighbors = [i+1 for i in neighbors]
        str_neighbors = ' '.join(str(ngh) for ngh in neighbors)
        print(f"Neighbors of Node {node+1}: {str_neighbors}")
        f.write(str_neighbors + "\n")


5237 6067
Neighbors of Node 1: 550 3191
Neighbors of Node 2: 3361 4506
Neighbors of Node 3: 319 4343
Neighbors of Node 4: 1287 2324
Neighbors of Node 5: 3934 5110
Neighbors of Node 6: 3774 4889
Neighbors of Node 7: 1719 2130
Neighbors of Node 8: 1693
Neighbors of Node 9: 218
Neighbors of Node 10: 3398 4417
Neighbors of Node 11: 2952 4773
Neighbors of Node 12: 1386 4295
Neighbors of Node 13: 297 1575
Neighbors of Node 14: 509 1486
Neighbors of Node 15: 365 4180
Neighbors of Node 16: 172 2179
Neighbors of Node 17: 741 822
Neighbors of Node 18: 2024 3386
Neighbors of Node 19: 2028 3513
Neighbors of Node 20: 720 2776
Neighbors of Node 21: 4664
Neighbors of Node 22: 392 1526 1926 2246 2575 3213 3912 4026 4044 5191
Neighbors of Node 23: 1271 4454
Neighbors of Node 24: 265 2994
Neighbors of Node 25: 3004 3298
Neighbors of Node 26: 2610 3127
Neighbors of Node 27: 3593 4287
Neighbors of Node 28: 886 3064
Neighbors of Node 29: 2190 2838 4655 4769
Neighbors of Node 30: 1203 2439
Neighbors of Node

In [65]:
gg = nx.DiGraph()

# Add nodes and edges (for demonstration purposes)
gg.add_edge(1, 5)
gg.add_edge(1, 3)
gg.add_edge(1, 2)
gg.add_edge(2, 1)
gg.add_edge(2, 3)
gg.add_edge(2, 4)
gg.add_edge(2, 1)
gg.add_edge(3, 5)
gg.add_edge(3, 4)
gg.add_edge(3, 2)
gg.add_edge(3, 1)
gg.add_edge(4, 2)
gg.add_edge(4, 3)
gg.add_edge(4, 6)
gg.add_edge(4, 7)
gg.add_edge(5, 1)
gg.add_edge(5, 3)
gg.add_edge(5, 6)
gg.add_edge(6, 5)
gg.add_edge(6, 4)
gg.add_edge(6, 7)
gg.add_edge(7, 6)
gg.add_edge(7, 4)


In [66]:
with open(output_name, 'w') as f:
    f.write(str(len(gg.nodes)) + " {}\n".format(gg.number_of_edges()))
    print(len(gg.nodes), gg.number_of_edges())
    for node, data in gg.nodes(data=True):
        neighbors = list(gg.neighbors(node))
        neighbors = [i for i in neighbors]
        str_neighbors = ' '.join(str(ngh) for ngh in neighbors)
        print(f"Neighbors of Node {node}: {str_neighbors}")

7 21
Neighbors of Node 1: 5 3 2
Neighbors of Node 5: 1 3
Neighbors of Node 3: 5 4 2 1
Neighbors of Node 2: 1 3 4
Neighbors of Node 4: 2 3 6 7
Neighbors of Node 6: 5 4 7
Neighbors of Node 7: 6 4
